# Proyecto II - Detección de Anomalías con Destilación de Modelos

**Curso de Inteligencia Artificial**  
**Escuela de Ingeniería en Computación**  
**Instituto Tecnológico de Costa Rica**

## Objetivo

Validar la hipótesis de que mediante destilación de modelos se pueden resolver tareas complejas con modelos más pequeños y eficientes.

Este notebook implementa tres modelos:
- **Modelo A**: CNN clasificador desde cero
- **Modelo B**: CNN clasificador con destilación teacher-student
- **Modelo C**: Autoencoder U-Net para reconstrucción


## 0. Configuración Inicial

### 0.1. Instalación de Dependencias


In [57]:
## 0. Configuración Inicial

### 0.1. Instalación de Dependencias
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install matplotlib numpy scikit-learn opencv-python pillow tqdm wandb pytorch-lightning hydra-core omegaconf torchmetrics


Looking in indexes: https://download.pytorch.org/whl/cu118


### 0.2. Montar Google Drive


In [58]:
### 0.2. Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 0.3. Imports y Configuración Inicial


In [59]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import resnet18
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
import hydra
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
import wandb
from torchmetrics import StructuralSimilarityIndexMeasure
from torchmetrics.classification import Accuracy
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"Pytorch Lightning version: {pl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


PyTorch version: 2.9.0+cu126
Pytorch Lightning version: 2.5.6
CUDA available: False


## 1. Definición de Arquitecturas de Modelos

En esta sección se definen las tres arquitecturas de modelos:

- **CNNClassifier** (Modelo A y B): Basado en ResNet-18 para las primeras 3 convoluciones
  - Modelo A: Entrenado desde cero (scratch)
  - Modelo B: Entrenado con destilación teacher-student (ResNet-18 como teacher)
  
- **UNetAutoencoder** (Modelo C): Autoencoder con skip connections para reconstrucción


In [60]:
# Cargar los modelos desde el archivo models.py
# En Colab, necesitamos copiar el contenido o importarlo desde Google Drive

# Opción 1: Si tienes models.py en Google Drive, puedes importarlo así:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/Proyecto-II')
# from models import CNNClassifier, UNetAutoencoder, BasicBlock

# Opción 2: Definir las clases directamente en el notebook (recomendado para Colab)
# Copiamos el contenido de models.py aquí

class BasicBlock(nn.Module):
    """Bloque básico de ResNet (2 convoluciones con skip connection)"""

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class CNNClassifier(nn.Module):
    """
    CNN Clasificador basado en ResNet-18 para las primeras 3 convoluciones
    Modelo A: Desde cero (scratch)
    Modelo B: Con destilación (distilled)
    """

    def __init__(self, num_classes=10, conv1_channels=64, conv2_channels=[64, 64],
                 conv3_channels=[128, 128], fc_hidden=512, dropout=0.5,
                 embedding_dim=256, model_type="scratch"):
        super(CNNClassifier, self).__init__()
        self.model_type = model_type
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim

        # conv1: Primera convolución (similar a ResNet-18)
        self.conv1 = nn.Conv2d(3, conv1_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(conv1_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # conv2_x: Bloques residuales
        self.conv2_x = self._make_layer(conv1_channels, conv2_channels[0], conv2_channels[1], num_blocks=2, stride=1)

        # conv3_x: Bloques residuales
        self.conv3_x = self._make_layer(conv2_channels[-1], conv3_channels[0], conv3_channels[1], num_blocks=2, stride=2)

        # Global Average Pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Clasificador
        self.fc = nn.Sequential(
            nn.Linear(conv3_channels[-1], fc_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(fc_hidden, num_classes)
        )

        # Capa para extraer embeddings (para detección de anomalías)
        self.embedding_layer = nn.Linear(conv3_channels[-1], embedding_dim)

    def _make_layer(self, in_channels, base_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(BasicBlock(in_channels, base_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(BasicBlock(base_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        # conv1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        # conv2_x
        x = self.conv2_x(x)

        # conv3_x
        x = self.conv3_x(x)

        # Global Average Pooling
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        # Embedding para detección de anomalías
        embedding = self.embedding_layer(x)

        # Clasificación
        logits = self.fc(x)

        return logits, embedding

    def get_embedding(self, x):
        """Extrae solo el embedding sin clasificación"""
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        embedding = self.embedding_layer(x)
        return embedding


class UNetAutoencoder(nn.Module):
    """
    Autoencoder U-Net con skip connections (Modelo C)
    Reutilizado de Tarea05
    """

    def __init__(self, input_channels=3, latent_dim=128, encoder_channels=None,
                 decoder_channels=None, embedding_dim=128):
        super(UNetAutoencoder, self).__init__()
        self.architecture = "unet_autoencoder"
        self.embedding_dim = embedding_dim

        if encoder_channels is None:
            encoder_channels = [64, 128, 256, 512]
        if decoder_channels is None:
            decoder_channels = [512, 256, 128, 64]

        # Encoder con skip connections
        self.encoder_blocks = nn.ModuleList()
        in_channels = input_channels

        for out_channels in encoder_channels:
            self.encoder_blocks.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1),
                    nn.ReLU(),
                    nn.BatchNorm2d(out_channels)
                )
            )
            in_channels = out_channels

        # Capa bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels, latent_dim, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        )

        # Decoder con skip connections
        self.decoder_blocks = nn.ModuleList()
        in_channels = latent_dim

        # Primera capa del decoder (sin skip connection)
        self.decoder_blocks.append(
            nn.Sequential(
                nn.ConvTranspose2d(in_channels, decoder_channels[0], kernel_size=4, stride=2, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(decoder_channels[0])
            )
        )
        in_channels = decoder_channels[0]

        # Resto de capas del decoder con skip connections
        for i, out_channels in enumerate(decoder_channels[1:], 1):
            self.decoder_blocks.append(
                nn.Sequential(
                    nn.ConvTranspose2d(in_channels * 2, out_channels, kernel_size=4, stride=2, padding=1),
                    nn.ReLU(),
                    nn.BatchNorm2d(out_channels)
                )
            )
            in_channels = out_channels

        # Capa final
        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(in_channels * 2, input_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

        # Capa para extraer embeddings
        self.embedding_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(latent_dim, embedding_dim)
        )

    def encode(self, x):
        """Extrae el vector latente de la entrada"""
        skip_connections = []
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x)
            skip_connections.append(x)

        x = self.bottleneck(x)
        return x, skip_connections

    def forward(self, x):
        # Encoder - guardar skip connections
        skip_connections = []
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x)
            skip_connections.append(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Decoder - usar skip connections
        x = self.decoder_blocks[0](x)

        for i, decoder_block in enumerate(self.decoder_blocks[1:], start=1):
            skip_idx = -i
            skip = skip_connections[skip_idx]

            if x.shape[2:] != skip.shape[2:]:
                x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=False)

            x = torch.cat([x, skip], dim=1)
            x = decoder_block(x)

        # Capa final
        skip = skip_connections[0]
        if x.shape[2:] != skip.shape[2:]:
            x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, skip], dim=1)
        x = self.final_layer(x)

        return x

    def get_embedding(self, x):
        """Extrae el embedding del espacio latente"""
        latent, _ = self.encode(x)
        embedding = self.embedding_layer(latent)
        return embedding

print("✓ Arquitecturas de modelos definidas correctamente")


✓ Arquitecturas de modelos definidas correctamente


## 2. Módulos de Pytorch Lightning

En esta sección se definen los módulos Lightning para entrenar los modelos:
- **CNNClassifierLightning**: Para Modelo A y B (con soporte para destilación)
- **AutoencoderLightning**: Para Modelo C
- **LossFunctions**: Funciones de pérdida (L1, L2, SSIM, SSIM_L1)


In [61]:
# Módulos Lightning - Copiamos el contenido de lightning_modules.py

class LossFunctions:
    """Funciones de pérdida para el entrenamiento"""

    @staticmethod
    def l1_loss(pred, target):
        return F.l1_loss(pred, target)

    @staticmethod
    def l2_loss(pred, target):
        return F.mse_loss(pred, target)

    @staticmethod
    def ssim_loss(pred, target):
        ssim = StructuralSimilarityIndexMeasure(data_range=2.0)
        ssim_val = ssim(pred, target)
        return 1 - ssim_val

    @staticmethod
    def ssim_l1_loss(pred, target, alpha=0.5):
        ssim = LossFunctions.ssim_loss(pred, target)
        l1 = LossFunctions.l1_loss(pred, target)
        return alpha * ssim + (1 - alpha) * l1


class CNNClassifierLightning(pl.LightningModule):
    """Módulo Lightning para entrenar CNN clasificadores (Modelo A y B)"""

    def __init__(self, model, num_classes=10, learning_rate=0.001, weight_decay=1e-5,
                 scheduler_config=None, model_type="scratch", teacher_model=None,
                 distillation_config=None):
        super().__init__()
        self.model = model
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.model_type = model_type
        self.scheduler_config = scheduler_config or {"name": "step", "step_size": 15, "gamma": 0.5}

        # Configuración de destilación (solo para Modelo B)
        self.distillation_config = distillation_config or {}
        self.teacher_model = teacher_model
        if model_type == "distilled" and teacher_model is None:
            # Cargar ResNet-18 pre-entrenado como teacher
            try:
                # Versión nueva de torchvision (weights)
                from torchvision.models import ResNet18_Weights
                self.teacher_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
            except:
                # Versión antigua (pretrained)
                self.teacher_model = resnet18(pretrained=True)
            self.teacher_model.fc = nn.Linear(self.teacher_model.fc.in_features, num_classes)
            self.teacher_model.eval()
            for param in self.teacher_model.parameters():
                param.requires_grad = False

        # Métricas
        self.train_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = Accuracy(task="multiclass", num_classes=num_classes)

        # Criterio de pérdida
        self.criterion = nn.CrossEntropyLoss()

        # Guardar hiperparámetros
        self.save_hyperparameters(ignore=['model', 'teacher_model'])

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits, embeddings = self(images)

        # Pérdida de clasificación
        loss = self.criterion(logits, labels)

        # Pérdida de destilación (solo para Modelo B)
        if self.model_type == "distilled" and self.teacher_model is not None:
            with torch.no_grad():
                teacher_logits = self.teacher_model(images)

            temperature = self.distillation_config.get("temperature", 4.0)
            alpha = self.distillation_config.get("alpha", 0.7)

            # Softmax con temperatura
            student_soft = F.log_softmax(logits / temperature, dim=1)
            teacher_soft = F.softmax(teacher_logits / temperature, dim=1)

            # Pérdida de destilación (KL divergence)
            distillation_loss = F.kl_div(student_soft, teacher_soft, reduction='batchmean') * (temperature ** 2)

            # Combinar pérdidas
            loss = alpha * distillation_loss + (1 - alpha) * loss

        # Logging
        self.log('train/loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train/acc', self.train_acc(logits, labels), on_step=True, on_epoch=True, prog_bar=True)
        self.log('train/learning_rate', self.optimizers().param_groups[0]['lr'], on_step=True)

        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits, embeddings = self(images)
        loss = self.criterion(logits, labels)

        # Logging
        self.log('val/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/acc', self.val_acc(logits, labels), on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits, embeddings = self(images)
        loss = self.criterion(logits, labels)

        # Logging
        self.log('test/loss', loss, on_step=False, on_epoch=True)
        self.log('test/acc', self.test_acc(logits, labels), on_step=False, on_epoch=True)

        return {'logits': logits, 'labels': labels, 'embeddings': embeddings}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

        scheduler_name = self.scheduler_config.get("name", "step")
        if scheduler_name == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer,
                step_size=self.scheduler_config.get("step_size", 15),
                gamma=self.scheduler_config.get("gamma", 0.5)
            )
        elif scheduler_name == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=self.scheduler_config.get("T_max", 50)
            )
        elif scheduler_name == "plateau":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                mode='min',
                factor=self.scheduler_config.get("factor", 0.5),
                patience=self.scheduler_config.get("patience", 5)
            )
        else:
            scheduler = None

        if scheduler is None:
            return optimizer
        else:
            # Para ReduceLROnPlateau, necesitamos incluir el monitor
            if scheduler_name == "plateau":
                return {
                    "optimizer": optimizer,
                    "lr_scheduler": {
                        "scheduler": scheduler,
                        "interval": "epoch",
                        "monitor": "val/loss"  # Métrica a monitorear para ReduceLROnPlateau
                    }
                }
            else:
                return {
                    "optimizer": optimizer,
                    "lr_scheduler": {
                        "scheduler": scheduler,
                        "interval": "epoch"
                    }
                }


class AutoencoderLightning(pl.LightningModule):
    """Módulo Lightning para entrenar autoencoders (Modelo C)"""

    def __init__(self, model, learning_rate=0.001, loss_function="L2", scheduler_config=None):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.loss_function = loss_function
        self.scheduler_config = scheduler_config or {"name": "step", "step_size": 15, "gamma": 0.5}

        # Inicializar función de pérdida
        if loss_function == "L1":
            self.criterion = LossFunctions.l1_loss
        elif loss_function == "L2":
            self.criterion = LossFunctions.l2_loss
        elif loss_function == "SSIM":
            self.criterion = LossFunctions.ssim_loss
        elif loss_function == "SSIM_L1":
            self.criterion = LossFunctions.ssim_l1_loss
        else:
            raise ValueError(f"Función de pérdida no reconocida: {loss_function}")

        # Métricas
        self.ssim_metric = StructuralSimilarityIndexMeasure(data_range=2.0)

        # Guardar hiperparámetros
        self.save_hyperparameters(ignore=['model'])

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x = batch[0] if isinstance(batch, tuple) else batch
        x_recon = self(x)
        loss = self.criterion(x_recon, x)

        # Logging
        self.log('train/loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train/learning_rate', self.optimizers().param_groups[0]['lr'], on_step=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x = batch[0] if isinstance(batch, tuple) else batch
        x_recon = self(x)
        loss = self.criterion(x_recon, x)

        # Calcular SSIM
        ssim_val = self.ssim_metric(x_recon, x)

        # Logging
        self.log('val/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/ssim', ssim_val, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x = batch[0] if isinstance(batch, tuple) else batch
        x_recon = self(x)
        loss = self.criterion(x_recon, x)

        # Extraer embeddings
        embeddings = self.model.get_embedding(x)

        # Logging
        self.log('test/loss', loss, on_step=False, on_epoch=True)

        return {'reconstructions': x_recon, 'originals': x, 'embeddings': embeddings}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)

        scheduler_name = self.scheduler_config.get("name", "step")
        if scheduler_name == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer,
                step_size=self.scheduler_config.get("step_size", 15),
                gamma=self.scheduler_config.get("gamma", 0.5)
            )
        else:
            scheduler = None

        if scheduler is None:
            return optimizer
        else:
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "epoch"
                }
            }

print("✓ Módulos Lightning definidos correctamente")


✓ Módulos Lightning definidos correctamente


In [62]:
# Configuración de rutas (similar a Tarea05)
# Definir ruta base de Google Drive
DRIVE_BASE_PATH = '/content/drive/MyDrive/Colab Notebooks'
drive_conf_dir = os.path.join(DRIVE_BASE_PATH, 'conf')

# Crear directorio base si no existe
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

print(f"🔍 Configurando para usar Google Drive...")
print(f"   Ruta base: {DRIVE_BASE_PATH}")
print(f"   Configuración: {drive_conf_dir}")

# Verificar si existe en Google Drive
config_file_drive = os.path.join(drive_conf_dir, 'config.yaml')

if not os.path.exists(config_file_drive):
    # Si no existe, copiar desde el directorio actual (si existe)
    current_conf_dir = './conf'
    if os.path.exists(current_conf_dir):
        print(f"⚠️ No se encontró configuración en Google Drive")
        print(f"   Copiando desde directorio actual: {current_conf_dir}")
        import shutil
        if os.path.exists(drive_conf_dir):
            shutil.rmtree(drive_conf_dir)
        shutil.copytree(current_conf_dir, drive_conf_dir)
        print(f"  ✓ Configuración copiada a Google Drive")
    else:
        print(f"⚠️ No se encontró configuración. Creando estructura básica...")
        # Crear estructura básica (similar a Tarea05 pero adaptada)
        os.makedirs(drive_conf_dir, exist_ok=True)
        os.makedirs(os.path.join(drive_conf_dir, 'model'), exist_ok=True)
        os.makedirs(os.path.join(drive_conf_dir, 'trainer'), exist_ok=True)
        os.makedirs(os.path.join(drive_conf_dir, 'logger'), exist_ok=True)

        # Crear archivos de configuración básicos
        # (El usuario deberá ajustar las rutas según su configuración)
        print(f"  ✓ Estructura creada. Por favor, ajusta los archivos de configuración.")

# Copiar temporalmente al directorio actual para Hydra
import shutil
current_dir = os.getcwd()
conf_dir_temp = os.path.join(current_dir, 'conf')

if os.path.exists(conf_dir_temp):
    shutil.rmtree(conf_dir_temp)

if os.path.exists(drive_conf_dir):
    shutil.copytree(drive_conf_dir, conf_dir_temp)
    print(f"  ✓ Configuración copiada temporalmente a: {conf_dir_temp}")
    config_path_for_hydra = 'conf'
else:
    print(f"⚠️ No se encontró configuración. Usando configuración por defecto.")
    config_path_for_hydra = None

# Autenticación de Weights & Biases
wandb.login()

# Limpiar Hydra si ya está inicializado
from hydra.core.global_hydra import GlobalHydra
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
    print("✓ Limpiando instancia previa de Hydra")

# Inicializar Hydra
if config_path_for_hydra and os.path.exists(os.path.join(conf_dir_temp, 'config.yaml')):
    try:
        print(f"✓ Inicializando Hydra con config_path='{config_path_for_hydra}'")
        hydra.initialize(config_path=config_path_for_hydra, version_base=None, job_name="notebook")
        print("✓ Hydra inicializado correctamente")

        # Registrar las clases del notebook en el resolver de Hydra
        import types
        notebook_models = types.ModuleType('notebook_models')
        notebook_models.CNNClassifier = CNNClassifier
        notebook_models.UNetAutoencoder = UNetAutoencoder
        sys.modules['notebook_models'] = notebook_models

        print("✓ Clases del notebook registradas para Hydra")

        # Cargar configuración
        cfg = hydra.compose(config_name="config")
        print("✓ Configuración cargada:")
        print(OmegaConf.to_yaml(cfg))

    except Exception as e:
        print(f"❌ Error al inicializar Hydra: {e}")
        print("   Continuando sin Hydra (configuración manual)")
        cfg = None
else:
    print("⚠️ No se encontró configuración de Hydra. Usando valores por defecto.")
    cfg = None

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✓ Usando dispositivo: {device}')

# Si no hay configuración de Hydra, usar valores por defecto
if cfg is None:
    print("⚠️ Usando configuración por defecto (sin Hydra)")
    # Valores por defecto
    DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/Proyecto2-IA/dataset'
    CATEGORIES = ["bottle", "cable", "capsule", "grid", "metal_nut", "pill", "screw", "tile", "transistor", "zipper"]
    IMAGE_SIZE = 128
    BATCH_SIZE = 32
    NUM_WORKERS = 2
else:
    DATASET_PATH = cfg.dataset.path
    CATEGORIES = cfg.dataset.categories
    IMAGE_SIZE = cfg.dataset.image_size
    BATCH_SIZE = cfg.dataset.batch_size
    NUM_WORKERS = cfg.dataset.num_workers

print(f'✓ Ruta base de Google Drive configurada: {DRIVE_BASE_PATH}')

# Validar que el dataset existe antes de continuar
print(f"\n🔍 Validando dataset...")
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(
        f"❌ ERROR: No se encontró el dataset en la ruta: {DATASET_PATH}\n"
        f"   Por favor, asegúrate de que el dataset MVTec AD esté en esa ubicación."
    )

print(f"  ✓ Dataset encontrado en: {DATASET_PATH}")

# Validar que todas las categorías existen
missing_categories = []
for category in CATEGORIES:
    category_path = os.path.join(DATASET_PATH, category)
    if not os.path.exists(category_path):
        missing_categories.append(category)
    else:
        # Verificar que tiene las carpetas train y test
        train_path = os.path.join(category_path, 'train')
        test_path = os.path.join(category_path, 'test')
        if not os.path.exists(train_path):
            print(f"  ⚠️ Advertencia: {category} no tiene carpeta 'train'")
        if not os.path.exists(test_path):
            print(f"  ⚠️ Advertencia: {category} no tiene carpeta 'test'")

if missing_categories:
    raise FileNotFoundError(
        f"❌ ERROR: Las siguientes categorías no se encontraron en el dataset:\n"
        f"   {missing_categories}\n"
        f"   Ruta del dataset: {DATASET_PATH}\n"
        f"   Categorías esperadas: {CATEGORIES}"
    )

print(f"  ✓ Todas las {len(CATEGORIES)} categorías encontradas: {CATEGORIES}")
print(f"✓ Validación del dataset completada\n")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


🔍 Configurando para usar Google Drive...
   Ruta base: /content/drive/MyDrive/Colab Notebooks
   Configuración: /content/drive/MyDrive/Colab Notebooks/conf
  ✓ Configuración copiada temporalmente a: /content/conf
✓ Limpiando instancia previa de Hydra
✓ Inicializando Hydra con config_path='conf'
✓ Hydra inicializado correctamente
✓ Clases del notebook registradas para Hydra
✓ Configuración cargada:
model:
  _target_: notebook_models.CNNClassifier
  model_type: scratch
  architecture: cnn_classifier_scratch
  conv1_channels: 64
  conv2_channels:
  - 64
  - 64
  conv3_channels:
  - 128
  - 128
  num_classes: 10
  fc_hidden: 512
  dropout: 0.5
  embedding_dim: 256
trainer:
  max_epochs: 50
  learning_rate: 0.001
  weight_decay: 1.0e-05
  optimizer: adam
  momentum: 0.9
  scheduler:
    name: step
    step_size: 15
    gamma: 0.5
    patience: 5
    factor: 0.5
  early_stopping:
    enabled: true
    monitor: val/loss
    mode: min
    patience: 10
    min_delta: 0.001
  checkpoint:
    sav

## 4. Carga y Preprocesamiento de Datos

En esta sección se carga el dataset MVTec AD con 10 clases. **Importante**: Solo se usan datos sin defectos ('good') para el entrenamiento.


In [63]:
# DataModule para MVTec AD - Copiamos el contenido de data_module.py

class AnomalyDataset(Dataset):
    """Dataset para cargar imágenes de entrenamiento y prueba"""

    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.labels is not None:
            return image, self.labels[idx]
        return image


def load_dataset_paths(category_path, split='train', only_good=True):
    """Carga las rutas de las imágenes del dataset"""
    paths = []
    labels = []
    split_path = os.path.join(category_path, split)

    if split == 'train' and only_good:
        # Solo imágenes 'good' en entrenamiento
        good_path = os.path.join(split_path, 'good')
        if os.path.exists(good_path):
            for img_file in os.listdir(good_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    paths.append(os.path.join(good_path, img_file))
                    labels.append(0)  # Se actualizará con el índice de categoría
    else:
        # En test, cargar todas las clases (good y anomalías)
        if os.path.exists(split_path):
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    for img_file in os.listdir(class_path):
                        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                            paths.append(os.path.join(class_path, img_file))
                            # Label: 0 para 'good', 1 para anomalías
                            labels.append(0 if class_name == 'good' else 1)

    return paths, labels


class MVTecDataModule(pl.LightningDataModule):
    """DataModule para MVTec AD con múltiples categorías"""

    def __init__(self, dataset_path, categories, image_size=128, batch_size=32,
                 num_workers=2, train_split=0.8):
        super().__init__()
        self.dataset_path = dataset_path
        self.categories = categories
        self.image_size = image_size
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_split = train_split

        # Transformaciones (normalización a [-1, 1] para compatibilidad con Tanh)
        self.train_transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # [-1, 1]
        ])

        self.val_transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.test_transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.train_paths = []
        self.train_labels = []
        self.val_paths = []
        self.val_labels = []
        self.test_paths = []
        self.test_labels = []

    def setup(self, stage=None):
        """Carga las rutas de las imágenes para todas las categorías"""
        """Carga las rutas de las imágenes para todas las categorías"""
        # Validar que el dataset_path existe
        if not os.path.exists(self.dataset_path):
            raise FileNotFoundError(
                f"❌ ERROR: No se encontró el dataset en: {self.dataset_path}\n"
                f"   Por favor, verifica la ruta del dataset."
            )

        all_train_paths = []
        all_train_labels = []
        all_test_paths = []
        all_test_labels = []

        # Cargar datos de todas las categorías
        for cat_idx, category in enumerate(self.categories):
            category_path = os.path.join(self.dataset_path, category)

            # Validar que la categoría existe
            if not os.path.exists(category_path):
                raise FileNotFoundError(
                    f"❌ ERROR: Categoría '{category}' no encontrada en: {category_path}"
                )

            # Entrenamiento (solo 'good')
            train_paths, _ = load_dataset_paths(category_path, split='train', only_good=True)
            if len(train_paths) == 0:
                raise ValueError(
                    f"❌ ERROR: No se encontraron imágenes de entrenamiento para la categoría '{category}'\n"
                    f"   Ruta esperada: {os.path.join(category_path, 'train', 'good')}"
                )
            # Asignar label de categoría
            train_labels = [cat_idx] * len(train_paths)
            all_train_paths.extend(train_paths)
            all_train_labels.extend(train_labels)

            # Prueba (todas las clases)
            test_paths, test_labels = load_dataset_paths(category_path, split='test', only_good=False)
            if len(test_paths) == 0:
                raise ValueError(
                    f"❌ ERROR: No se encontraron imágenes de prueba para la categoría '{category}'\n"
                    f"   Ruta esperada: {os.path.join(category_path, 'test')}"
                )
            all_test_paths.extend(test_paths)
            all_test_labels.extend(test_labels)

        # Dividir entrenamiento en train y validation
        total_train = len(all_train_paths)
        train_size = int(self.train_split * total_train)
        val_size = total_train - train_size

        indices = torch.randperm(total_train).tolist()
        train_indices = indices[:train_size]
        val_indices = indices[train_size:]

        self.train_paths = [all_train_paths[i] for i in train_indices]
        self.train_labels = [all_train_labels[i] for i in train_indices]
        self.val_paths = [all_train_paths[i] for i in val_indices]
        self.val_labels = [all_train_labels[i] for i in val_indices]

        self.test_paths = all_test_paths
        self.test_labels = all_test_labels


        # Validación final: verificar que hay datos
        if len(self.train_paths) == 0:
            raise ValueError("❌ ERROR: No se encontraron imágenes de entrenamiento")
        if len(self.val_paths) == 0:
            raise ValueError("❌ ERROR: No se encontraron imágenes de validación")
        if len(self.test_paths) == 0:
            raise ValueError("❌ ERROR: No se encontraron imágenes de prueba")

        print(f"Train: {len(self.train_paths)} imágenes")
        print(f"Validation: {len(self.val_paths)} imágenes")
        print(f"Test: {len(self.test_paths)} imágenes")

    def train_dataloader(self):
        dataset = AnomalyDataset(self.train_paths, labels=self.train_labels, transform=self.train_transform)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        dataset = AnomalyDataset(self.val_paths, labels=self.val_labels, transform=self.val_transform)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        dataset = AnomalyDataset(self.test_paths, labels=self.test_labels, transform=self.test_transform)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

# Crear DataModule
data_module = MVTecDataModule(
    dataset_path=DATASET_PATH,
    categories=CATEGORIES,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_split=0.8
)

# Setup (cargar datos)
data_module.setup()

try:
    data_module.setup()
    print("✓ DataModule creado y configurado correctamente")

    # Validación adicional: verificar que hay datos
    if len(data_module.train_paths) == 0:
        raise ValueError("❌ ERROR: No se encontraron imágenes de entrenamiento")
    if len(data_module.val_paths) == 0:
        raise ValueError("❌ ERROR: No se encontraron imágenes de validación")
    if len(data_module.test_paths) == 0:
        raise ValueError("❌ ERROR: No se encontraron imágenes de prueba")

    print(f"  ✓ Datos cargados correctamente:")
    print(f"    - Entrenamiento: {len(data_module.train_paths)} imágenes")
    print(f"    - Validación: {len(data_module.val_paths)} imágenes")
    print(f"    - Prueba: {len(data_module.test_paths)} imágenes")

except Exception as e:
    print(f"\n❌ ERROR al configurar DataModule: {e}")
    print(f"   Por favor, verifica:")
    print(f"   1. Que el dataset esté en: {DATASET_PATH}")
    print(f"   2. Que todas las categorías existan: {CATEGORIES}")
    print(f"   3. Que cada categoría tenga carpetas 'train/good' y 'test'")
    raise

print("✓ DataModule creado y configurado correctamente")


Train: 1924 imágenes
Validation: 482 imágenes
Test: 1253 imágenes
Train: 1924 imágenes
Validation: 482 imágenes
Test: 1253 imágenes
✓ DataModule creado y configurado correctamente
  ✓ Datos cargados correctamente:
    - Entrenamiento: 1924 imágenes
    - Validación: 482 imágenes
    - Prueba: 1253 imágenes
✓ DataModule creado y configurado correctamente


## 5. Entrenamiento de Modelos

En esta sección se entrenan los tres modelos:
- **Modelo A**: CNN clasificador desde cero
- **Modelo B**: CNN clasificador con destilación
- **Modelo C**: Autoencoder U-Net

**Importante**:
- Cada modelo debe entrenarse con al menos 3 configuraciones diferentes de hiperparámetros
- Se usa EarlyStopping para evitar overfitting
- Todos los modelos se entrenan solo con datos sin defectos


In [64]:
# Función auxiliar para entrenar un modelo
def train_model(model_type, model_config, trainer_config, logger_config, experiment_name):
    """
    Entrena un modelo con la configuración especificada

    Args:
        model_type: "cnn_scratch", "cnn_distilled", o "unet"
        model_config: Configuración del modelo
        trainer_config: Configuración del entrenamiento
        logger_config: Configuración del logger
        experiment_name: Nombre del experimento para WandB
    """
    print(f"\n{'='*80}")
    print(f"ENTRENANDO: {experiment_name}")
    print(f"{'='*80}\n")

    # Separar parámetros del Lightning module de los del Trainer
    # Parámetros que van al Lightning module
    lightning_params = {
        'learning_rate': trainer_config.get('learning_rate', 0.001),
        'weight_decay': trainer_config.get('weight_decay', 1e-5),
        'scheduler_config': trainer_config.get('scheduler_config', {"name": "step", "step_size": 15, "gamma": 0.5})
    }

    # Para modelos con destilación
    if 'distillation_config' in trainer_config:
        lightning_params['distillation_config'] = trainer_config['distillation_config']

    # Crear modelo base
    if model_type == "cnn_scratch":
        base_model = CNNClassifier(
            num_classes=len(CATEGORIES),
            model_type="scratch",
            **model_config
        )
        lightning_model = CNNClassifierLightning(
            model=base_model,
            num_classes=len(CATEGORIES),
            model_type="scratch",
            **lightning_params
        )
    elif model_type == "cnn_distilled":
        base_model = CNNClassifier(
            num_classes=len(CATEGORIES),
            model_type="distilled",
            **model_config
        )
        lightning_model = CNNClassifierLightning(
            model=base_model,
            num_classes=len(CATEGORIES),
            model_type="distilled",
            **lightning_params
        )
    elif model_type == "unet":
        base_model = UNetAutoencoder(**model_config)
        # Para AutoencoderLightning, también necesitamos loss_function
        autoencoder_params = {
            'learning_rate': trainer_config.get('learning_rate', 0.001),
            'loss_function': trainer_config.get('loss_function', 'L2'),
            'scheduler_config': trainer_config.get('scheduler_config', {"name": "step", "step_size": 15, "gamma": 0.5})
        }
        lightning_model = AutoencoderLightning(
            model=base_model,
            **autoencoder_params
        )
    else:
        raise ValueError(f"Tipo de modelo no reconocido: {model_type}")

    # Configurar logger
    wandb_logger = WandbLogger(
        project=logger_config.get("project", "proyecto-ii-anomaly-detection"),
        name=experiment_name,
        config={
            "model_type": model_type,
            "model_config": model_config,
            "trainer_config": trainer_config
        },
        reinit=True
    )

    # Callbacks
    early_stopping = EarlyStopping(
        monitor=trainer_config.get("early_stopping", {}).get("monitor", "val/loss"),
        mode=trainer_config.get("early_stopping", {}).get("mode", "min"),
        patience=trainer_config.get("early_stopping", {}).get("patience", 10),
        min_delta=trainer_config.get("early_stopping", {}).get("min_delta", 0.001)
    )

    checkpoint_callback = ModelCheckpoint(
        monitor=trainer_config.get("checkpoint", {}).get("monitor", "val/loss"),
        mode=trainer_config.get("checkpoint", {}).get("mode", "min"),
        save_top_k=trainer_config.get("checkpoint", {}).get("save_top_k", 3),
        save_last=True,
        dirpath=os.path.join(DRIVE_BASE_PATH, 'checkpoints', experiment_name),
        filename=f'{experiment_name}-{{epoch:02d}}-{{val/loss:.4f}}'
    )

    lr_monitor = LearningRateMonitor(logging_interval='step')

    # Crear Trainer
    trainer = pl.Trainer(
        max_epochs=trainer_config.get("max_epochs", 50),
        accelerator='auto',
        devices=1,
        logger=wandb_logger,
        callbacks=[early_stopping, checkpoint_callback, lr_monitor],
        log_every_n_steps=10,
        enable_progress_bar=True,
        gradient_clip_val=trainer_config.get("gradient_clip_val", 1.0),
        accumulate_grad_batches=trainer_config.get("accumulate_grad_batches", 1)
    )

    # Entrenar

    # Validar que el data_module está configurado
    try:
        if not hasattr(data_module, 'train_paths') or len(data_module.train_paths) == 0:
            raise ValueError("❌ ERROR: El data_module no está configurado correctamente. Ejecuta data_module.setup() primero.")
    except AttributeError:
        raise ValueError("❌ ERROR: El data_module no está configurado. Ejecuta data_module.setup() primero.")

    trainer.fit(lightning_model, data_module)

    # Evaluar
    trainer.test(lightning_model, data_module)

    # Evaluar
    try:
        trainer.test(lightning_model, data_module)
    except Exception as e:
        print(f"⚠️ Advertencia: Error durante la evaluación: {e}")
        # Continuar aunque falle la evaluación

    return lightning_model, trainer, wandb_logger

print("✓ Función de entrenamiento definida")


✓ Función de entrenamiento definida


### 5.1. Entrenamiento del Modelo A (CNN desde cero)

Entrenar el modelo A con múltiples configuraciones de hiperparámetros (al menos 3).


In [65]:
# Configuraciones de hiperparámetros para Modelo A (al menos 3)
model_a_configs = [
    {
        "model_config": {
            "conv1_channels": 64,
            "conv2_channels": [64, 64],
            "conv3_channels": [128, 128],
            "fc_hidden": 512,
            "dropout": 0.5,
            "embedding_dim": 256
        },
        "trainer_config": {
            "learning_rate": 0.001,
            "weight_decay": 1e-5,
            "max_epochs": 50,
            "scheduler_config": {"name": "step", "step_size": 15, "gamma": 0.5},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_a_config1"
    },
    {
        "model_config": {
            "conv1_channels": 64,
            "conv2_channels": [64, 64],
            "conv3_channels": [128, 128],
            "fc_hidden": 256,
            "dropout": 0.3,
            "embedding_dim": 128
        },
        "trainer_config": {
            "learning_rate": 0.0005,
            "weight_decay": 1e-4,
            "max_epochs": 50,
            "scheduler_config": {"name": "cosine", "T_max": 50},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_a_config2"
    },
    {
        "model_config": {
            "conv1_channels": 64,
            "conv2_channels": [64, 64],
            "conv3_channels": [128, 128],
            "fc_hidden": 1024,
            "dropout": 0.7,
            "embedding_dim": 512
        },
        "trainer_config": {
            "learning_rate": 0.002,
            "weight_decay": 1e-6,
            "max_epochs": 50,
            "scheduler_config": {"name": "plateau", "factor": 0.5, "patience": 5},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_a_config3"
    }
]

# Entrenar Modelo A con todas las configuraciones
model_a_results = []
for config in model_a_configs:
    try:
        model, trainer, logger = train_model(
            model_type="cnn_scratch",
            model_config=config["model_config"],
            trainer_config=config["trainer_config"],
            logger_config={"project": "proyecto-ii-anomaly-detection"},
            experiment_name=config["experiment_name"]
        )
        model_a_results.append({
            "config": config["experiment_name"],
            "model": model,
            "trainer": trainer,
            "logger": logger
        })
        print(f"✓ {config['experiment_name']} completado\n")
    except Exception as e:
        print(f"❌ Error en {config['experiment_name']}: {e}\n")

print(f"✓ Modelo A: {len(model_a_results)} configuraciones entrenadas")


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores



ENTRENANDO: model_a_config1



INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | CNNClassifier      | 787 K  | train
1 | train_acc | MulticlassAccuracy | 0      | train
2 | val_acc   | MulticlassAccuracy | 0      | train
3 | test_acc  | MulticlassAccuracy | 0      | train
4 | criterion | CrossEntropyLoss   | 0      | train
---------------------------------------------------------
787 K     Trainable params
0         Non-trainable params
787 K     Total params
3.149     Total estimated model params size (MB)
43        Modules in train mode
0         Modules in eval mode


Train: 1924 imágenes
Validation: 482 imágenes
Test: 1253 imágenes


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.08938547223806381    │
│         test/loss         │     40.22410202026367     │
└───────────────────────────┴───────────────────────────┘

Train: 1924 imágenes
Validation: 482 imágenes
Test: 1253 imágenes


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.08938547223806381    │
│         test/loss         │     40.22410202026367     │
└───────────────────────────┴───────────────────────────┘

✓ model_a_config3 completado

✓ Modelo A: 3 configuraciones entrenadas


### 5.2. Entrenamiento del Modelo B (CNN con destilación)

Entrenar el modelo B con múltiples configuraciones de hiperparámetros (al menos 3).


In [66]:
# Configuraciones de hiperparámetros para Modelo B (al menos 3)
model_b_configs = [
    {
        "model_config": {
            "conv1_channels": 64,
            "conv2_channels": [64, 64],
            "conv3_channels": [128, 128],
            "fc_hidden": 512,
            "dropout": 0.5,
            "embedding_dim": 256
        },
        "trainer_config": {
            "learning_rate": 0.001,
            "weight_decay": 1e-5,
            "max_epochs": 50,
            "scheduler_config": {"name": "step", "step_size": 15, "gamma": 0.5},
            "distillation_config": {"temperature": 4.0, "alpha": 0.7},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_b_config1"
    },
    {
        "model_config": {
            "conv1_channels": 64,
            "conv2_channels": [64, 64],
            "conv3_channels": [128, 128],
            "fc_hidden": 256,
            "dropout": 0.3,
            "embedding_dim": 128
        },
        "trainer_config": {
            "learning_rate": 0.0005,
            "weight_decay": 1e-4,
            "max_epochs": 50,
            "scheduler_config": {"name": "cosine", "T_max": 50},
            "distillation_config": {"temperature": 5.0, "alpha": 0.8},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_b_config2"
    },
    {
        "model_config": {
            "conv1_channels": 64,
            "conv2_channels": [64, 64],
            "conv3_channels": [128, 128],
            "fc_hidden": 1024,
            "dropout": 0.7,
            "embedding_dim": 512
        },
        "trainer_config": {
            "learning_rate": 0.002,
            "weight_decay": 1e-6,
            "max_epochs": 50,
            "scheduler_config": {"name": "plateau", "factor": 0.5, "patience": 5},
            "distillation_config": {"temperature": 3.0, "alpha": 0.6},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_b_config3"
    }
]

# Entrenar Modelo B con todas las configuraciones
model_b_results = []
for config in model_b_configs:
    try:
        # Crear modelo con destilación
        base_model = CNNClassifier(
            num_classes=len(CATEGORIES),
            model_type="distilled",
            **config["model_config"]
        )
        lightning_model = CNNClassifierLightning(
            model=base_model,
            num_classes=len(CATEGORIES),
            model_type="distilled",
            distillation_config=config["trainer_config"].get("distillation_config", {}),
            **{k: v for k, v in config["trainer_config"].items() if k != "distillation_config"}
        )

        # Configurar logger
        wandb_logger = WandbLogger(
            project="proyecto-ii-anomaly-detection",
            name=config["experiment_name"],
            config={
                "model_type": "cnn_distilled",
                "model_config": config["model_config"],
                "trainer_config": config["trainer_config"]
            },
            reinit=True
        )

        # Callbacks
        early_stopping = EarlyStopping(
            monitor=config["trainer_config"].get("early_stopping", {}).get("monitor", "val/loss"),
            mode=config["trainer_config"].get("early_stopping", {}).get("mode", "min"),
            patience=config["trainer_config"].get("early_stopping", {}).get("patience", 10),
            min_delta=config["trainer_config"].get("early_stopping", {}).get("min_delta", 0.001)
        )

        checkpoint_callback = ModelCheckpoint(
            monitor=config["trainer_config"].get("checkpoint", {}).get("monitor", "val/loss"),
            mode=config["trainer_config"].get("checkpoint", {}).get("mode", "min"),
            save_top_k=config["trainer_config"].get("checkpoint", {}).get("save_top_k", 3),
            save_last=True,
            dirpath=os.path.join(DRIVE_BASE_PATH, 'checkpoints', config["experiment_name"]),
            filename=f'{config["experiment_name"]}-{{epoch:02d}}-{{val/loss:.4f}}'
        )

        lr_monitor = LearningRateMonitor(logging_interval='step')

        # Crear Trainer
        trainer = pl.Trainer(
            max_epochs=config["trainer_config"].get("max_epochs", 50),
            accelerator='auto',
            devices=1,
            logger=wandb_logger,
            callbacks=[early_stopping, checkpoint_callback, lr_monitor],
            log_every_n_steps=10,
            enable_progress_bar=True
        )

        # Entrenar
        trainer.fit(lightning_model, data_module)
        trainer.test(lightning_model, data_module)

        model_b_results.append({
            "config": config["experiment_name"],
            "model": lightning_model,
            "trainer": trainer,
            "logger": wandb_logger
        })
        print(f"✓ {config['experiment_name']} completado\n")
    except Exception as e:
        print(f"❌ Error en {config['experiment_name']}: {e}\n")

print(f"✓ Modelo B: {len(model_b_results)} configuraciones entrenadas")


❌ Error en model_b_config1: CNNClassifierLightning.__init__() got an unexpected keyword argument 'max_epochs'

❌ Error en model_b_config2: CNNClassifierLightning.__init__() got an unexpected keyword argument 'max_epochs'

❌ Error en model_b_config3: CNNClassifierLightning.__init__() got an unexpected keyword argument 'max_epochs'

✓ Modelo B: 0 configuraciones entrenadas


In [67]:
# Configuraciones de hiperparámetros para Modelo C (al menos 3)
model_c_configs = [
    {
        "model_config": {
            "input_channels": 3,
            "latent_dim": 128,
            "encoder_channels": [64, 128, 256, 512],
            "decoder_channels": [512, 256, 128, 64],
            "embedding_dim": 128
        },
        "trainer_config": {
            "learning_rate": 0.001,
            "loss_function": "L2",
            "max_epochs": 50,
            "scheduler_config": {"name": "step", "step_size": 15, "gamma": 0.5},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_c_config1_l2"
    },
    {
        "model_config": {
            "input_channels": 3,
            "latent_dim": 256,
            "encoder_channels": [64, 128, 256, 512],
            "decoder_channels": [512, 256, 128, 64],
            "embedding_dim": 256
        },
        "trainer_config": {
            "learning_rate": 0.0005,
            "loss_function": "SSIM_L1",
            "max_epochs": 50,
            "scheduler_config": {"name": "cosine", "T_max": 50},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_c_config2_ssim_l1"
    },
    {
        "model_config": {
            "input_channels": 3,
            "latent_dim": 64,
            "encoder_channels": [32, 64, 128, 256],
            "decoder_channels": [256, 128, 64, 32],
            "embedding_dim": 64
        },
        "trainer_config": {
            "learning_rate": 0.002,
            "loss_function": "L1",
            "max_epochs": 50,
            "scheduler_config": {"name": "plateau", "factor": 0.5, "patience": 5},
            "early_stopping": {"monitor": "val/loss", "mode": "min", "patience": 10, "min_delta": 0.001},
            "checkpoint": {"monitor": "val/loss", "mode": "min", "save_top_k": 3}
        },
        "experiment_name": "model_c_config3_l1"
    }
]

# Entrenar Modelo C con todas las configuraciones
model_c_results = []
for config in model_c_configs:
    try:
        model, trainer, logger = train_model(
            model_type="unet",
            model_config=config["model_config"],
            trainer_config=config["trainer_config"],
            logger_config={"project": "proyecto-ii-anomaly-detection"},
            experiment_name=config["experiment_name"]
        )
        model_c_results.append({
            "config": config["experiment_name"],
            "model": model,
            "trainer": trainer,
            "logger": logger
        })
        print(f"✓ {config['experiment_name']} completado\n")
    except Exception as e:
        print(f"❌ Error en {config['experiment_name']}: {e}\n")

print(f"✓ Modelo C: {len(model_c_results)} configuraciones entrenadas")



ENTRENANDO: model_c_config1_l2



INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type                              | Params | Mode 
--------------------------------------------------------------------------
0 | model       | UNetAutoencoder                   | 10.4 M | train
1 | ssim_metric | _StructuralSimilarityIndexMeasure | 0      | train
--------------------------------------------------------------------------
10.4 M    Trainable params
0         Non-trainable params
10.4 M    Total params
41.545    Total estimated model params size (MB)
46        Modules in train mode
0         Modules in eval mode


Train: 1924 imágenes
Validation: 482 imágenes
Test: 1253 imágenes


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

❌ Error en model_c_config1_l2: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)



ENTRENANDO: model_c_config2_ssim_l1



...

❌ Error en model_c_config2_ssim_l1: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)



ENTRENANDO: model_c_config3_l1



INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type                              | Params | Mode 
--------------------------------------------------------------------------
0 | model       | UNetAutoencoder                   | 2.6 M  | train
1 | ssim_metric | _StructuralSimilarityIndexMeasure | 0      | train
--------------------------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.402    Total estimated model params size (MB)
46        Modules in train mode
0         Modules in eval mode


Train: 1924 imágenes
Validation: 482 imágenes
Test: 1253 imágenes


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

❌ Error en model_c_config3_l1: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


✓ Modelo C: 0 configuraciones entrenadas


## 6. Evaluación de Anomalías

Una vez entrenados los modelos, se calculan las representaciones latentes (embeddings) de las imágenes del conjunto de **validación o entrenamiento** (solo datos normales) para estimar la distribución normal. Luego se aplica esa distribución al conjunto de prueba para identificar datos anómalos.

### Proceso de Evaluación:

1. **Estimación de la distribución normal**:
   - Extraer embeddings del conjunto de validación/entrenamiento (solo datos normales)
   - Calcular media μ = (1/N) Σ z_i
   - Calcular matriz de covarianza Σ = (1/(N-1)) Σ (z_i - μ)(z_i - μ)^T
   - Modelar como distribución gaussiana multivariada N(μ, Σ)

2. **Cálculo de distancias en conjunto de prueba**:
   - Extraer embeddings del conjunto de prueba
   - Calcular distancias usando la distribución normal estimada
   - Determinar umbral usando percentil de distancias normales de validación

**Métodos de evaluación**:
- **Distancia de Mahalanobis**: d = sqrt((z - μ)^T Σ^(-1) (z - μ))
- **Distancia Euclidiana**: d = ||z - μ||
- **Reconstruction Loss**: Error de reconstrucción para autoencoders


In [68]:
# Funciones de evaluación - Copiamos el contenido de evaluation.py

from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv

def calculate_mahalanobis_distance(embeddings, mean, cov):
    """
    Calcula la distancia de Mahalanobis para cada embedding.

    Distancia de Mahalanobis: d = sqrt((z - μ)^T Σ^(-1) (z - μ))

    Args:
        embeddings: Array numpy de shape (N, d) con embeddings
        mean: Vector media de shape (d,)
        cov: Matriz de covarianza de shape (d, d)

    Returns:
        distances: Array numpy de shape (N,) con distancias de Mahalanobis
    """
    if embeddings is None or len(embeddings) == 0:
        raise ValueError("❌ ERROR: embeddings no puede estar vacío")
    if mean is None:
        raise ValueError("❌ ERROR: mean no puede ser None")
    if cov is None:
        raise ValueError("❌ ERROR: cov no puede ser None")

    try:
        # Regularización para evitar singularidad
        cov_reg = cov + np.eye(cov.shape[0]) * 1e-6

        # Calcular inversa de la matriz de covarianza
        try:
            cov_inv = inv(cov_reg)
        except np.linalg.LinAlgError as e:
            raise ValueError(f"❌ ERROR: No se pudo invertir la matriz de covarianza: {e}") from e

        # Calcular distancias
        distances = []
        for emb in embeddings:
            diff = emb - mean
            try:
                dist = np.sqrt(diff @ cov_inv @ diff.T)
                if np.isnan(dist) or np.isinf(dist):
                    print(f"  ⚠️ Advertencia: Distancia inválida detectada, usando 0")
                    dist = 0.0
                distances.append(dist)
            except Exception as e:
                print(f"  ⚠️ Advertencia: Error calculando distancia: {e}, usando 0")
                distances.append(0.0)

        return np.array(distances)

    except Exception as e:
        raise RuntimeError(f"❌ ERROR al calcular distancias de Mahalanobis: {e}") from e


def extract_embeddings(model, dataloader, device):
    """
    Extrae embeddings de un dataloader usando el modelo entrenado

    Args:
        model: Modelo entrenado
        dataloader: DataLoader con las imágenes
        device: Dispositivo (cuda/cpu)

    Returns:
        all_embeddings: Array numpy con todos los embeddings
        all_labels: Array numpy con todas las etiquetas (o None)
        all_reconstructions: Lista de reconstrucciones (para autoencoders)
        all_originals: Lista de imágenes originales (para autoencoders)
    """
    if model is None:
        raise ValueError("❌ ERROR: El modelo no puede ser None")

    if dataloader is None:
        raise ValueError("❌ ERROR: El dataloader no puede ser None")

    model.eval()
    all_embeddings = []
    all_labels = []
    all_reconstructions = []
    all_originals = []

    try:
        with torch.no_grad():
            for batch_idx, batch in enumerate(dataloader):
                try:
                    if isinstance(batch, tuple):
                        images, labels = batch
                    else:
                        images = batch
                        labels = None

                    if images is None or images.numel() == 0:
                        print(f"  ⚠️ Advertencia: Batch {batch_idx} está vacío, saltando...")
                        continue

                    images = images.to(device)

                    # Extraer embeddings
                    try:
                        if hasattr(model, 'get_embedding'):
                            embeddings = model.get_embedding(images)
                        elif hasattr(model, 'model') and hasattr(model.model, 'get_embedding'):
                            embeddings = model.model.get_embedding(images)
                        else:
                            if hasattr(model, 'model'):
                                logits, embeddings = model.model(images)
                            else:
                                logits, embeddings = model(images)
                    except Exception as e:
                        print(f"  ❌ Error extrayendo embeddings del batch {batch_idx}: {e}")
                        raise

                    all_embeddings.append(embeddings.cpu().numpy())

                    if labels is not None:
                        all_labels.append(labels.cpu().numpy())

                    # Para autoencoders, guardar reconstrucciones
                    if hasattr(model, 'model') and hasattr(model.model, 'forward'):
                        try:
                            reconstructions = model.model(images)
                            all_reconstructions.append(reconstructions.cpu().numpy())
                            all_originals.append(images.cpu().numpy())
                        except Exception as e:
                            # Si falla la reconstrucción, continuar sin ella
                            pass

                except Exception as e:
                    print(f"  ⚠️ Error procesando batch {batch_idx}: {e}")
                    continue

        if len(all_embeddings) == 0:
            raise ValueError("❌ ERROR: No se pudieron extraer embeddings. El dataloader podría estar vacío.")

        all_embeddings = np.concatenate(all_embeddings, axis=0)
        all_labels = np.concatenate(all_labels, axis=0) if all_labels else None

        return all_embeddings, all_labels, all_reconstructions, all_originals

    except Exception as e:
        raise RuntimeError(f"❌ ERROR al extraer embeddings: {e}") from e


def estimate_normal_distribution(normal_embeddings):
    """
    Estima la distribución normal (gaussiana multivariada) a partir de embeddings normales.

    Calcula la media μ y la matriz de covarianza Σ:
    μ = (1/N) Σ z_i
    Σ = (1/(N-1)) Σ (z_i - μ)(z_i - μ)^T

    Args:
        normal_embeddings: Array numpy de shape (N, d) con embeddings normales

    Returns:
        mean: Vector media de shape (d,)
        cov: Matriz de covarianza de shape (d, d)
    """
    if normal_embeddings is None or len(normal_embeddings) == 0:
        raise ValueError("❌ ERROR: normal_embeddings no puede estar vacío")

    if len(normal_embeddings.shape) != 2:
        raise ValueError(f"❌ ERROR: normal_embeddings debe ser 2D, pero tiene shape {normal_embeddings.shape}")

    if len(normal_embeddings) < 2:
        raise ValueError(f"❌ ERROR: Se necesitan al menos 2 muestras para calcular covarianza, pero hay {len(normal_embeddings)}")

    try:
        # Media: μ = (1/N) Σ z_i
        mean = np.mean(normal_embeddings, axis=0)

        # Matriz de covarianza: Σ = (1/(N-1)) Σ (z_i - μ)(z_i - μ)^T
        # np.cov usa (N-1) como denominador por defecto
        cov = np.cov(normal_embeddings.T)

        # Validar que la matriz de covarianza es válida
        if np.any(np.isnan(cov)) or np.any(np.isinf(cov)):
            raise ValueError("❌ ERROR: La matriz de covarianza contiene NaN o Inf")

        return mean, cov

    except Exception as e:
        raise RuntimeError(f"❌ ERROR al estimar distribución normal: {e}") from e


def evaluate_anomaly_detection(model, normal_dataloader, test_dataloader, device, method="mahalanobis", percentile=95):
    """
    Evalúa la detección de anomalías siguiendo el proceso correcto:
    1. Estima la distribución normal usando el conjunto de validación/entrenamiento (solo datos normales)
    2. Aplica esa distribución al conjunto de prueba para detectar anomalías

    Args:
        model: Modelo entrenado
        normal_dataloader: Dataloader con datos normales (validación o entrenamiento)
        test_dataloader: Dataloader con datos de prueba (normales y anómalos)
        device: Dispositivo (cuda/cpu)
        method: Método de evaluación ("mahalanobis", "euclidean", "reconstruction_loss")
        percentile: Percentil para determinar el umbral (default: 95)

    Returns:
        results: Diccionario con resultados de la evaluación
    """
    # Validación de parámetros
    if model is None:
        raise ValueError("❌ ERROR: El modelo no puede ser None")
    if normal_dataloader is None:
        raise ValueError("❌ ERROR: normal_dataloader no puede ser None")
    if test_dataloader is None:
        raise ValueError("❌ ERROR: test_dataloader no puede ser None")
    if method not in ["mahalanobis", "euclidean", "reconstruction_loss"]:
        raise ValueError(f"❌ ERROR: Método '{method}' no reconocido. Use: 'mahalanobis', 'euclidean', o 'reconstruction_loss'")
    if not (0 < percentile <= 100):
        raise ValueError(f"❌ ERROR: Percentil debe estar entre 0 y 100, pero es {percentile}")

    try:
        # Paso 1: Extraer embeddings del conjunto normal (validación/entrenamiento)
        print("📊 Estimando distribución normal a partir del conjunto de validación/entrenamiento...")
        normal_embeddings, _, normal_reconstructions, normal_originals = extract_embeddings(
            model, normal_dataloader, device
        )

        if len(normal_embeddings) == 0:
            raise ValueError("❌ ERROR: No se pudieron extraer embeddings del conjunto normal")

        # Estimar distribución normal: μ y Σ
        mean, cov = estimate_normal_distribution(normal_embeddings)
        print(f"  ✓ Media (μ) calculada: shape {mean.shape}")
        print(f"  ✓ Matriz de covarianza (Σ) calculada: shape {cov.shape}")

        # Paso 2: Extraer embeddings del conjunto de prueba
        print("\n📊 Extrayendo embeddings del conjunto de prueba...")
        test_embeddings, test_labels, test_reconstructions, test_originals = extract_embeddings(
            model, test_dataloader, device
        )

        if len(test_embeddings) == 0:
            raise ValueError("❌ ERROR: No se pudieron extraer embeddings del conjunto de prueba")

        # Separar embeddings normales y anómalos del conjunto de prueba
        if test_labels is not None:
            test_normal_embeddings = test_embeddings[test_labels == 0]
            test_anomaly_embeddings = test_embeddings[test_labels == 1]

            # Calcular distancias usando la distribución normal estimada
            try:
                if method == "mahalanobis":
                    # Distancia de Mahalanobis: d = sqrt((z - μ)^T Σ^(-1) (z - μ))
                    if len(test_normal_embeddings) > 0:
                        test_normal_distances = calculate_mahalanobis_distance(test_normal_embeddings, mean, cov)
                    else:
                        test_normal_distances = np.array([])

                    if len(test_anomaly_embeddings) > 0:
                        test_anomaly_distances = calculate_mahalanobis_distance(test_anomaly_embeddings, mean, cov)
                    else:
                        test_anomaly_distances = np.array([])

                elif method == "euclidean":
                    # Distancia euclidiana: d = ||z - μ||
                    if len(test_normal_embeddings) > 0:
                        test_normal_distances = np.linalg.norm(test_normal_embeddings - mean, axis=1)
                    else:
                        test_normal_distances = np.array([])

                    if len(test_anomaly_embeddings) > 0:
                        test_anomaly_distances = np.linalg.norm(test_anomaly_embeddings - mean, axis=1)
                    else:
                        test_anomaly_distances = np.array([])

                elif method == "reconstruction_loss":
                    if len(test_reconstructions) > 0 and len(test_originals) > 0:
                        test_reconstructions = np.concatenate(test_reconstructions, axis=0)
                        test_originals = np.concatenate(test_originals, axis=0)
                        test_normal_recon = test_reconstructions[test_labels == 0]
                        test_normal_orig = test_originals[test_labels == 0]
                        test_anomaly_recon = test_reconstructions[test_labels == 1]
                        test_anomaly_orig = test_originals[test_labels == 1]

                        if len(test_normal_recon) > 0:
                            test_normal_distances = np.mean((test_normal_recon - test_normal_orig) ** 2, axis=(1, 2, 3))
                        else:
                            test_normal_distances = np.array([])

                        if len(test_anomaly_recon) > 0:
                            test_anomaly_distances = np.mean((test_anomaly_recon - test_anomaly_orig) ** 2, axis=(1, 2, 3))
                        else:
                            test_anomaly_distances = np.array([])
                    else:
                        raise ValueError("❌ ERROR: Reconstruction loss requiere reconstrucciones. Asegúrate de usar un modelo autoencoder.")
            except Exception as e:
                raise RuntimeError(f"❌ ERROR al calcular distancias con método '{method}': {e}") from e

            # Determinar umbral usando percentil de las distancias normales del conjunto de validación
            # Primero calculamos distancias de los datos normales de validación
            try:
                if method == "mahalanobis":
                    validation_normal_distances = calculate_mahalanobis_distance(normal_embeddings, mean, cov)
                elif method == "euclidean":
                    validation_normal_distances = np.linalg.norm(normal_embeddings - mean, axis=1)
                elif method == "reconstruction_loss":
                    if len(normal_reconstructions) > 0 and len(normal_originals) > 0:
                        normal_reconstructions = np.concatenate(normal_reconstructions, axis=0)
                        normal_originals = np.concatenate(normal_originals, axis=0)
                        validation_normal_distances = np.mean((normal_reconstructions - normal_originals) ** 2, axis=(1, 2, 3))
                    else:
                        raise ValueError("❌ ERROR: Reconstruction loss requiere reconstrucciones del conjunto normal")

                if len(validation_normal_distances) == 0:
                    raise ValueError("❌ ERROR: No se pudieron calcular distancias de validación")

                # Umbral basado en percentil de distancias normales de validación
                threshold = np.percentile(validation_normal_distances, percentile)
                print(f"\n📏 Umbral calculado (percentil {percentile}): {threshold:.4f}")

            except Exception as e:
                raise RuntimeError(f"❌ ERROR al calcular umbral: {e}") from e

            # Clasificar datos de prueba
            if len(test_normal_distances) == 0 and len(test_anomaly_distances) == 0:
                raise ValueError("❌ ERROR: No hay distancias para clasificar")

            all_distances = np.concatenate([test_normal_distances, test_anomaly_distances])
            predictions = (all_distances > threshold).astype(int)
            true_labels = np.concatenate([np.zeros_like(test_normal_distances), np.ones_like(test_anomaly_distances)])

            # Calcular métricas
            try:
                auc_roc = roc_auc_score(true_labels, all_distances)
                auc_pr = average_precision_score(true_labels, all_distances)
            except Exception as e:
                print(f"  ⚠️ Advertencia: Error al calcular métricas: {e}")
                auc_roc = 0.0
                auc_pr = 0.0

            results = {
                'method': method,
                'threshold': threshold,
                'auc_roc': auc_roc,
                'auc_pr': auc_pr,
                'normal_distances': test_normal_distances,
                'anomaly_distances': test_anomaly_distances,
                'all_distances': all_distances,
                'predictions': predictions,
                'true_labels': true_labels,
                'mean': mean,
                'cov': cov,
                'validation_normal_distances': validation_normal_distances
            }
        else:
            # Si no hay labels, solo retornar estadísticas
            mean = np.mean(normal_embeddings, axis=0)
            cov = np.cov(normal_embeddings.T)
            results = {
                'method': method,
                'mean': mean,
                'cov': cov,
                'normal_embeddings': normal_embeddings,
                'test_embeddings': test_embeddings
            }

        return results

    except Exception as e:
        error_msg = f"❌ ERROR en evaluate_anomaly_detection: {e}"
        print(error_msg)
        raise RuntimeError(error_msg) from e

print("✓ Funciones de evaluación definidas")


✓ Funciones de evaluación definidas


### 6.1. Evaluación de todos los modelos entrenados

Evaluar cada modelo entrenado usando distancia de Mahalanobis y otras métricas.


In [69]:
# Evaluar todos los modelos entrenados
all_evaluation_results = []

# Evaluar Modelo A
print("Evaluando Modelo A...")
for result in model_a_results:
    try:
        eval_result = evaluate_anomaly_detection(
            model=result["model"],
            normal_dataloader=data_module.val_dataloader(),  # Usar validación para estimar distribución normal
            test_dataloader=data_module.test_dataloader(),    # Usar test para evaluar
            device=device,
            method="mahalanobis",
            percentile=95
        )
        all_evaluation_results.append({
            "model_type": "Modelo A",
            "config": result["config"],
            "auc_roc": eval_result["auc_roc"],
            "auc_pr": eval_result["auc_pr"],
            "threshold": eval_result["threshold"]
        })
        print(f"  {result['config']}: AUC-ROC={eval_result['auc_roc']:.4f}, AUC-PR={eval_result['auc_pr']:.4f}")
    except Exception as e:
        print(f"  ❌ Error evaluando {result['config']}: {e}")

# Evaluar Modelo B
print("\nEvaluando Modelo B...")
for result in model_b_results:
    try:
        eval_result = evaluate_anomaly_detection(
            model=result["model"],
            normal_dataloader=data_module.val_dataloader(),  # Usar validación para estimar distribución normal
            test_dataloader=data_module.test_dataloader(),    # Usar test para evaluar
            device=device,
            method="mahalanobis",
            percentile=95
        )
        all_evaluation_results.append({
            "model_type": "Modelo B",
            "config": result["config"],
            "auc_roc": eval_result["auc_roc"],
            "auc_pr": eval_result["auc_pr"],
            "threshold": eval_result["threshold"]
        })
        print(f"  {result['config']}: AUC-ROC={eval_result['auc_roc']:.4f}, AUC-PR={eval_result['auc_pr']:.4f}")
    except Exception as e:
        print(f"  ❌ Error evaluando {result['config']}: {e}")

# Evaluar Modelo C
print("\nEvaluando Modelo C...")
for result in model_c_results:
    try:
        # Para autoencoders, también evaluar con reconstruction_loss
        eval_result_mah = evaluate_anomaly_detection(
            model=result["model"],
            normal_dataloader=data_module.val_dataloader(),  # Usar validación para estimar distribución normal
            test_dataloader=data_module.test_dataloader(),    # Usar test para evaluar
            device=device,
            method="mahalanobis",
            percentile=95
        )
        eval_result_recon = evaluate_anomaly_detection(
            model=result["model"],
            normal_dataloader=data_module.val_dataloader(),  # Usar validación para estimar distribución normal
            test_dataloader=data_module.test_dataloader(),    # Usar test para evaluar
            device=device,
            method="reconstruction_loss",
            percentile=95
        )
        all_evaluation_results.append({
            "model_type": "Modelo C",
            "config": result["config"],
            "auc_roc_mah": eval_result_mah["auc_roc"],
            "auc_pr_mah": eval_result_mah["auc_pr"],
            "auc_roc_recon": eval_result_recon["auc_roc"],
            "auc_pr_recon": eval_result_recon["auc_pr"]
        })
        print(f"  {result['config']}: Mahalanobis AUC-ROC={eval_result_mah['auc_roc']:.4f}, Recon AUC-ROC={eval_result_recon['auc_roc']:.4f}")
    except Exception as e:
        print(f"  ❌ Error evaluando {result['config']}: {e}")

print(f"\n✓ Evaluación completada: {len(all_evaluation_results)} modelos evaluados")

# Identificar los 3 mejores modelos
if all_evaluation_results:
    # Ordenar por AUC-ROC (usar el mejor método para cada modelo)
    sorted_results = sorted(
        all_evaluation_results,
        key=lambda x: max(x.get("auc_roc", 0), x.get("auc_roc_mah", 0), x.get("auc_roc_recon", 0)),
        reverse=True
    )
    best_3_models = sorted_results[:3]
    print(f"\n🏆 Top 3 modelos:")
    for i, result in enumerate(best_3_models, 1):
        best_auc = max(result.get("auc_roc", 0), result.get("auc_roc_mah", 0), result.get("auc_roc_recon", 0))
        print(f"  {i}. {result['model_type']} - {result['config']}: AUC-ROC={best_auc:.4f}")


Evaluando Modelo A...
📊 Estimando distribución normal a partir del conjunto de validación/entrenamiento...
  ⚠️ Error procesando batch 0: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 1: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 2: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 3: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 4: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 5: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 6: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 7: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 8: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 9: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 10: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 11: 'list' object has no attribute 'numel'
  ⚠️ Error procesando batch 12: 'list' object has n

## 7. Cuantización de Modelos

Convertir los **3 mejores modelos** (según AUC-ROC) a modelos cuantizados y realizar una comparación completa:

### Comparaciones realizadas:
1. **Tamaño del modelo**: Comparación de tamaño en MB y ratio de compresión
2. **Latencia en respuesta**: Tiempo de inferencia promedio (100 iteraciones)
3. **Rendimiento**: Comparación de métricas de detección de anomalías:
   - AUC-ROC (Area Under ROC Curve)
   - AUC-PR (Area Under Precision-Recall Curve)
   - Diferencia y porcentaje de retención de rendimiento


In [70]:
# Funciones de cuantización - Copiamos el contenido de evaluation.py

def quantize_model(model, method="dynamic"):
    """
    Cuantiza un modelo PyTorch
    """
    model.eval()

    if method == "dynamic":
        quantized_model = torch.quantization.quantize_dynamic(
            model, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8
        )
    elif method == "static":
        # Para cuantización estática, necesitaríamos un dataset de calibración
        quantized_model = torch.quantization.quantize_dynamic(
            model, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8
        )
    else:
        raise ValueError(f"Método de cuantización no reconocido: {method}")

    return quantized_model


def compare_model_sizes(original_model, quantized_model):
    """Compara el tamaño de modelos original y cuantizado"""
    def get_model_size(model):
        param_size = sum(p.numel() * p.element_size() for p in model.parameters())
        buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
        return param_size + buffer_size

    original_size = get_model_size(original_model)
    quantized_size = get_model_size(quantized_model)

    return {
        'original_size_mb': original_size / (1024 * 1024),
        'quantized_size_mb': quantized_size / (1024 * 1024),
        'compression_ratio': original_size / quantized_size if quantized_size > 0 else 0
    }


# Cuantizar los 3 mejores modelos
quantization_results = []

# Verificar que best_3_models existe y tiene contenido
if 'best_3_models' not in globals() or not best_3_models:
    print("⚠️ ERROR: No se encontraron los mejores modelos.")
    print("   Por favor, ejecuta primero la celda de evaluación (Sección 6) para identificar los mejores modelos.")
    print("   La variable 'best_3_models' debe estar definida antes de ejecutar la cuantización.")
else:
    print("Cuantizando los 3 mejores modelos...\n")

    for i, best_model_info in enumerate(best_3_models, 1):
        print(f"Modelo {i}: {best_model_info['model_type']} - {best_model_info['config']}")

        # Encontrar el modelo correspondiente
        model_to_quantize = None
        if best_model_info['model_type'] == "Modelo A":
            for result in model_a_results:
                if result['config'] == best_model_info['config']:
                    model_to_quantize = result['model'].model
                    break
        elif best_model_info['model_type'] == "Modelo B":
            for result in model_b_results:
                if result['config'] == best_model_info['config']:
                    model_to_quantize = result['model'].model
                    break
        elif best_model_info['model_type'] == "Modelo C":
            for result in model_c_results:
                if result['config'] == best_model_info['config']:
                    model_to_quantize = result['model'].model
                    break

        if model_to_quantize is None:
            print(f"  ⚠️ No se encontró el modelo\n")
            continue

        try:
            # Cuantizar
            quantized_model = quantize_model(model_to_quantize, method="dynamic")

            # Comparar tamaños
            size_comparison = compare_model_sizes(model_to_quantize, quantized_model)

            # Medir latencia (inferencia) - promedio sobre múltiples iteraciones
            model_to_quantize.eval()
            quantized_model.eval()

            # Crear un batch de prueba
            test_batch = next(iter(data_module.test_dataloader()))
            if isinstance(test_batch, tuple):
                test_images = test_batch[0][:1].to(device)
            else:
                test_images = test_batch[:1].to(device)

            # Latencia original - promedio sobre 100 iteraciones
            import time
            model_to_quantize = model_to_quantize.to(device)
            latencies_original = []
            with torch.no_grad():
                for _ in range(100):
                    start_time = time.time()
                    if hasattr(model_to_quantize, 'get_embedding'):
                        _ = model_to_quantize.get_embedding(test_images)
                    else:
                        _ = model_to_quantize(test_images)
                    latencies_original.append((time.time() - start_time) * 1000)  # ms
            original_latency = np.mean(latencies_original)

            # Latencia cuantizado - promedio sobre 100 iteraciones
            quantized_model = quantized_model.to(device)
            latencies_quantized = []
            with torch.no_grad():
                for _ in range(100):
                    start_time = time.time()
                    if hasattr(quantized_model, 'get_embedding'):
                        _ = quantized_model.get_embedding(test_images)
                    else:
                        _ = quantized_model(test_images)
                    latencies_quantized.append((time.time() - start_time) * 1000)  # ms
            quantized_latency = np.mean(latencies_quantized)

            # Evaluar rendimiento (precisión/accuracy) del modelo original
            print("  📊 Evaluando rendimiento del modelo original...")
            original_performance = None
            try:
                # Usar el modelo Lightning completo para evaluación
                if best_model_info['model_type'] == "Modelo A":
                    for result in model_a_results:
                        if result['config'] == best_model_info['config']:
                            lightning_model_original = result['model']
                            break
                elif best_model_info['model_type'] == "Modelo B":
                    for result in model_b_results:
                        if result['config'] == best_model_info['config']:
                            lightning_model_original = result['model']
                            break
                elif best_model_info['model_type'] == "Modelo C":
                    for result in model_c_results:
                        if result['config'] == best_model_info['config']:
                            lightning_model_original = result['model']
                            break

                # Evaluar con distancia de Mahalanobis
                eval_original = evaluate_anomaly_detection(
                    model=lightning_model_original,
                    normal_dataloader=data_module.val_dataloader(),
                    test_dataloader=data_module.test_dataloader(),
                    device=device,
                    method="mahalanobis",
                    percentile=95
                )
                original_performance = {
                    'auc_roc': eval_original['auc_roc'],
                    'auc_pr': eval_original['auc_pr']
                }
                print(f"    ✓ AUC-ROC: {original_performance['auc_roc']:.4f}, AUC-PR: {original_performance['auc_pr']:.4f}")
            except Exception as e:
                print(f"    ⚠️ Error evaluando modelo original: {e}")
                original_performance = {'auc_roc': 0.0, 'auc_pr': 0.0}

            # Evaluar rendimiento del modelo cuantizado
            print("  📊 Evaluando rendimiento del modelo cuantizado...")
            quantized_performance = None
            try:
                # Crear un wrapper Lightning para el modelo cuantizado
                if best_model_info['model_type'] in ["Modelo A", "Modelo B"]:
                    # Para modelos de clasificación, necesitamos el wrapper Lightning
                    quantized_lightning = CNNClassifierLightning(
                        model=quantized_model,
                        num_classes=len(CATEGORIES),
                        model_type="scratch" if best_model_info['model_type'] == "Modelo A" else "distilled"
                    )
                else:  # Modelo C
                    quantized_lightning = AutoencoderLightning(
                        model=quantized_model,
                        loss_function="L2"
                    )

                # Evaluar con distancia de Mahalanobis
                eval_quantized = evaluate_anomaly_detection(
                    model=quantized_lightning,
                    normal_dataloader=data_module.val_dataloader(),
                    test_dataloader=data_module.test_dataloader(),
                    device=device,
                    method="mahalanobis",
                    percentile=95
                )
                quantized_performance = {
                    'auc_roc': eval_quantized['auc_roc'],
                    'auc_pr': eval_quantized['auc_pr']
                }
                print(f"    ✓ AUC-ROC: {quantized_performance['auc_roc']:.4f}, AUC-PR: {quantized_performance['auc_pr']:.4f}")
            except Exception as e:
                print(f"    ⚠️ Error evaluando modelo cuantizado: {e}")
                quantized_performance = {'auc_roc': 0.0, 'auc_pr': 0.0}

            # Calcular diferencia de rendimiento
            performance_diff_auc_roc = original_performance['auc_roc'] - quantized_performance['auc_roc']
            performance_diff_auc_pr = original_performance['auc_pr'] - quantized_performance['auc_pr']
            performance_retention_auc_roc = (quantized_performance['auc_roc'] / original_performance['auc_roc'] * 100) if original_performance['auc_roc'] > 0 else 0
            performance_retention_auc_pr = (quantized_performance['auc_pr'] / original_performance['auc_pr'] * 100) if original_performance['auc_pr'] > 0 else 0

            quantization_results.append({
                "model_type": best_model_info['model_type'],
                "config": best_model_info['config'],
                "original_size_mb": size_comparison['original_size_mb'],
                "quantized_size_mb": size_comparison['quantized_size_mb'],
                "compression_ratio": size_comparison['compression_ratio'],
                "original_latency_ms": original_latency,
                "quantized_latency_ms": quantized_latency,
                "speedup": original_latency / quantized_latency if quantized_latency > 0 else 0,
                "original_auc_roc": original_performance['auc_roc'],
                "quantized_auc_roc": quantized_performance['auc_roc'],
                "original_auc_pr": original_performance['auc_pr'],
                "quantized_auc_pr": quantized_performance['auc_pr'],
                "performance_diff_auc_roc": performance_diff_auc_roc,
                "performance_diff_auc_pr": performance_diff_auc_pr,
                "performance_retention_auc_roc": performance_retention_auc_roc,
                "performance_retention_auc_pr": performance_retention_auc_pr
            })

            print(f"\n  📊 COMPARACIÓN DE RESULTADOS:")
            print(f"  {'='*60}")
            print(f"  Tamaño:")
            print(f"    Original: {size_comparison['original_size_mb']:.2f} MB")
            print(f"    Cuantizado: {size_comparison['quantized_size_mb']:.2f} MB")
            print(f"    Compresión: {size_comparison['compression_ratio']:.2f}x")
            print(f"\n  Latencia (promedio sobre 100 iteraciones):")
            print(f"    Original: {original_latency:.2f} ms")
            print(f"    Cuantizado: {quantized_latency:.2f} ms")
            print(f"    Speedup: {original_latency / quantized_latency if quantized_latency > 0 else 0:.2f}x")
            print(f"\n  Rendimiento (AUC-ROC):")
            print(f"    Original: {original_performance['auc_roc']:.4f}")
            print(f"    Cuantizado: {quantized_performance['auc_roc']:.4f}")
            print(f"    Diferencia: {performance_diff_auc_roc:+.4f}")
            print(f"    Retención: {performance_retention_auc_roc:.2f}%")
            print(f"\n  Rendimiento (AUC-PR):")
            print(f"    Original: {original_performance['auc_pr']:.4f}")
            print(f"    Cuantizado: {quantized_performance['auc_pr']:.4f}")
            print(f"    Diferencia: {performance_diff_auc_pr:+.4f}")
            print(f"    Retención: {performance_retention_auc_pr:.2f}%")
            print(f"  {'='*60}\n")

        except Exception as e:
            print(f"  ❌ Error al cuantizar: {e}\n")

    print(f"✓ Cuantización completada: {len(quantization_results)} modelos cuantizados")

    # Mostrar resumen comparativo completo
    if quantization_results:
        print("\n" + "="*80)
        print("RESUMEN COMPARATIVO DE CUANTIZACIÓN")
        print("="*80)
        print("\nComparación de los 3 mejores modelos: Original vs Cuantizado\n")

        for i, result in enumerate(quantization_results, 1):
            print(f"{i}. {result['model_type']} - {result['config']}")
            print(f"   {'-'*70}")
            print(f"   Tamaño:")
            print(f"     Original: {result['original_size_mb']:.2f} MB → Cuantizado: {result['quantized_size_mb']:.2f} MB")
            print(f"     Compresión: {result['compression_ratio']:.2f}x")
            print(f"   Latencia:")
            print(f"     Original: {result['original_latency_ms']:.2f} ms → Cuantizado: {result['quantized_latency_ms']:.2f} ms")
            print(f"     Speedup: {result['speedup']:.2f}x")
            print(f"   Rendimiento (AUC-ROC):")
            print(f"     Original: {result['original_auc_roc']:.4f} → Cuantizado: {result['quantized_auc_roc']:.4f}")
            print(f"     Diferencia: {result['performance_diff_auc_roc']:+.4f} ({result['performance_retention_auc_roc']:.2f}% retención)")
            print(f"   Rendimiento (AUC-PR):")
            print(f"     Original: {result['original_auc_pr']:.4f} → Cuantizado: {result['quantized_auc_pr']:.4f}")
            print(f"     Diferencia: {result['performance_diff_auc_pr']:+.4f} ({result['performance_retention_auc_pr']:.2f}% retención)")
            print()

        # Resumen estadístico
        print("="*80)
        print("RESUMEN ESTADÍSTICO")
        print("="*80)
        avg_compression = np.mean([r['compression_ratio'] for r in quantization_results])
        avg_speedup = np.mean([r['speedup'] for r in quantization_results])
        avg_retention_auc_roc = np.mean([r['performance_retention_auc_roc'] for r in quantization_results])
        avg_retention_auc_pr = np.mean([r['performance_retention_auc_pr'] for r in quantization_results])

        print(f"\nPromedio de compresión: {avg_compression:.2f}x")
        print(f"Promedio de speedup: {avg_speedup:.2f}x")
        print(f"Retención promedio de rendimiento (AUC-ROC): {avg_retention_auc_roc:.2f}%")
        print(f"Retención promedio de rendimiento (AUC-PR): {avg_retention_auc_pr:.2f}%")
        print("="*80)


⚠️ ERROR: No se encontraron los mejores modelos.
   Por favor, ejecuta primero la celda de evaluación (Sección 6) para identificar los mejores modelos.
   La variable 'best_3_models' debe estar definida antes de ejecutar la cuantización.


## 8. Análisis de Outliers mediante DBSCAN Clustering

Una vez identificado el mejor modelo de detección de anomalías, se utilizan sus embeddings para realizar un análisis adicional mediante técnicas de agrupamiento no supervisado.

**DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) es un método basado en densidad que permite:
- Identificar regiones de alta concentración en el espacio latente
- Detectar puntos aislados (outliers/anomalías) que se encuentran en zonas de baja densidad

### Proceso de Análisis:

1. **Extracción de embeddings**: Se extraen los embeddings del mejor modelo para cada imagen del conjunto de prueba
2. **Reducción de dimensionalidad**:
   - **PCA**: Para facilitar el procesamiento y reducir dimensionalidad
   - **t-SNE**: Para visualización 2D y separación estructural
3. **Aplicación de DBSCAN**: Los puntos etiquetados como ruido (-1) representan potenciales anomalías
4. **Análisis visual y cuantitativo**: Comparación de resultados DBSCAN con ground truth


In [71]:
# Funciones DBSCAN - Copiamos el contenido de evaluation.py

def dbscan_analysis(embeddings, eps=0.5, min_samples=5, use_pca=True, pca_components=50,
                    use_tsne=True, tsne_components=2, tsne_perplexity=30):
    """
    Análisis DBSCAN para detección de outliers mediante clustering basado en densidad.

    Proceso:
    1. Reducción de dimensionalidad con PCA (opcional)
    2. Aplicación de DBSCAN para identificar clusters y outliers
    3. Reducción adicional con t-SNE para visualización 2D

    Args:
        embeddings: Array numpy de shape (N, d) con embeddings
        eps: Distancia máxima entre muestras para formar un cluster
        min_samples: Número mínimo de muestras para formar un cluster
        use_pca: Si usar PCA para reducción de dimensionalidad
        pca_components: Número de componentes PCA
        use_tsne: Si usar t-SNE para visualización 2D
        tsne_components: Dimensiones de salida de t-SNE (típicamente 2)
        tsne_perplexity: Perplejidad para t-SNE

    Returns:
        results: Diccionario con resultados del análisis
    """
    print(f"📊 Iniciando análisis DBSCAN...")
    print(f"  Embeddings originales: shape {embeddings.shape}")

    # Reducción de dimensionalidad con PCA
    if use_pca and embeddings.shape[1] > pca_components:
        print(f"  Aplicando PCA: {embeddings.shape[1]} → {pca_components} dimensiones")
        pca = PCA(n_components=pca_components)
        embeddings_reduced = pca.fit_transform(embeddings)
        explained_variance = np.sum(pca.explained_variance_ratio_)
        print(f"  ✓ Varianza explicada por PCA: {explained_variance:.4f} ({explained_variance*100:.2f}%)")
    else:
        embeddings_reduced = embeddings
        pca = None
        explained_variance = 1.0
        print(f"  ⚠️ PCA no aplicado (use_pca=False o dimensión ya es {embeddings.shape[1]})")

    # Aplicar DBSCAN
    print(f"\n  Aplicando DBSCAN (eps={eps}, min_samples={min_samples})...")
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(embeddings_reduced)

    # Identificar outliers (ruido)
    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = list(clusters).count(-1)
    n_in_clusters = len(clusters) - n_noise

    print(f"  ✓ DBSCAN completado:")
    print(f"    - Clusters encontrados: {n_clusters}")
    print(f"    - Puntos en clusters: {n_in_clusters} ({n_in_clusters/len(clusters)*100:.2f}%)")
    print(f"    - Outliers (ruido): {n_noise} ({n_noise/len(clusters)*100:.2f}%)")

    # Reducción para visualización con t-SNE
    embeddings_2d = None
    if use_tsne:
        print(f"\n  Aplicando t-SNE para visualización 2D...")
        perplexity = min(tsne_perplexity, len(embeddings_reduced) - 1)
        if perplexity > 0:
            tsne = TSNE(n_components=tsne_components, random_state=42, perplexity=perplexity)
            embeddings_2d = tsne.fit_transform(embeddings_reduced)
            print(f"  ✓ t-SNE completado: {embeddings_reduced.shape[1]} → {tsne_components} dimensiones")
        else:
            print(f"  ⚠️ Perplexity inválida ({perplexity}), saltando t-SNE")
    else:
        print(f"  ⚠️ t-SNE deshabilitado (use_tsne=False)")

    results = {
        'clusters': clusters,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'embeddings_reduced': embeddings_reduced,
        'embeddings_2d': embeddings_2d,
        'pca': pca,
        'explained_variance': explained_variance
    }

    return results


def visualize_dbscan_results(dbscan_results, labels=None, save_path=None):
    """
    Visualiza los resultados de DBSCAN de forma completa.

    Muestra:
    1. Clustering DBSCAN (clusters y outliers)
    2. Comparación con ground truth labels
    3. Análisis de distribución de outliers vs normales
    """
    clusters = dbscan_results['clusters']
    embeddings_2d = dbscan_results['embeddings_2d']

    if embeddings_2d is None:
        print("⚠️ No hay visualización 2D disponible (t-SNE no se aplicó)")
        return

    # Crear figura con múltiples subplots para análisis completo
    fig = plt.figure(figsize=(18, 6))
    axes = fig.subplots(1, 3)

    # Visualización por clusters
    unique_clusters = set(clusters)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_clusters)))

    for cluster, color in zip(unique_clusters, colors):
        if cluster == -1:
            # Ruido (outliers)
            mask = clusters == cluster
            axes[0].scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
                          c='black', marker='x', s=50, label='Outliers', alpha=0.6)
        else:
            mask = clusters == cluster
            axes[0].scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
                          c=[color], s=50, label=f'Cluster {cluster}', alpha=0.6)

    axes[0].set_title(f'DBSCAN Clustering (Clusters: {dbscan_results["n_clusters"]}, Outliers: {dbscan_results["n_noise"]})')
    axes[0].set_xlabel('t-SNE Component 1')
    axes[0].set_ylabel('t-SNE Component 2')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Visualización por labels (si están disponibles)
    if labels is not None:
        normal_mask = labels == 0
        anomaly_mask = labels == 1

        axes[1].scatter(embeddings_2d[normal_mask, 0], embeddings_2d[normal_mask, 1],
                      c='green', s=50, label='Normal', alpha=0.6)
        axes[1].scatter(embeddings_2d[anomaly_mask, 0], embeddings_2d[anomaly_mask, 1],
                      c='red', s=50, label='Anomaly', alpha=0.6)

        axes[1].set_title('Ground Truth Labels')
        axes[1].set_xlabel('t-SNE Component 1')
        axes[1].set_ylabel('t-SNE Component 2')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        # Visualización adicional: Outliers DBSCAN vs Ground Truth
        outlier_mask = clusters == -1
        in_cluster_mask = clusters != -1

        # Colores: verde=normal, rojo=anomalía, tamaño grande=outlier DBSCAN
        normal_outliers = (outlier_mask) & (normal_mask)
        anomaly_outliers = (outlier_mask) & (anomaly_mask)
        normal_in_cluster = (in_cluster_mask) & (normal_mask)
        anomaly_in_cluster = (in_cluster_mask) & (anomaly_mask)

        axes[2].scatter(embeddings_2d[normal_in_cluster, 0], embeddings_2d[normal_in_cluster, 1],
                       c='lightgreen', s=30, label='Normal (en cluster)', alpha=0.5, marker='o')
        axes[2].scatter(embeddings_2d[normal_outliers, 0], embeddings_2d[normal_outliers, 1],
                       c='green', s=150, label='Normal (outlier DBSCAN)', alpha=0.8, marker='x', linewidths=2)
        axes[2].scatter(embeddings_2d[anomaly_in_cluster, 0], embeddings_2d[anomaly_in_cluster, 1],
                       c='lightcoral', s=30, label='Anomalía (en cluster)', alpha=0.5, marker='o')
        axes[2].scatter(embeddings_2d[anomaly_outliers, 0], embeddings_2d[anomaly_outliers, 1],
                       c='red', s=150, label='Anomalía (outlier DBSCAN)', alpha=0.8, marker='x', linewidths=2)

        axes[2].set_title('DBSCAN Outliers vs Ground Truth')
        axes[2].set_xlabel('t-SNE Component 1')
        axes[2].set_ylabel('t-SNE Component 2')
        axes[2].legend(loc='best', fontsize=8)
        axes[2].grid(True, alpha=0.3)
    else:
        # Si no hay labels, solo mostrar clustering
        axes[1].axis('off')
        axes[2].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"  ✓ Visualización guardada en: {save_path}")

    plt.show()

print("✓ Funciones DBSCAN definidas")


✓ Funciones DBSCAN definidas


### 8.1. Análisis DBSCAN del mejor modelo

Aplicar DBSCAN al mejor modelo identificado para análisis de outliers:

1. **Extracción de embeddings**: Se extraen embeddings del conjunto de prueba usando el mejor modelo
2. **Reducción de dimensionalidad**: PCA y t-SNE para facilitar visualización y procesamiento
3. **Clustering DBSCAN**: Identificación de clusters y outliers (puntos de baja densidad)
4. **Análisis visual**: Visualización de clusters, outliers y comparación con ground truth
5. **Análisis cuantitativo**: Métricas de clasificación comparando outliers DBSCAN con ground truth


In [72]:
# Seleccionar el mejor modelo para análisis DBSCAN
if best_3_models:
    best_model_info = best_3_models[0]
    print(f"Analizando con el mejor modelo: {best_model_info['model_type']} - {best_model_info['config']}\n")

    # Encontrar el modelo
    best_model = None
    if best_model_info['model_type'] == "Modelo A":
        for result in model_a_results:
            if result['config'] == best_model_info['config']:
                best_model = result['model']
                break
    elif best_model_info['model_type'] == "Modelo B":
        for result in model_b_results:
            if result['config'] == best_model_info['config']:
                best_model = result['model']
                break
    elif best_model_info['model_type'] == "Modelo C":
        for result in model_c_results:
            if result['config'] == best_model_info['config']:
                best_model = result['model']
                break

    if best_model is not None:
        # Extraer embeddings del conjunto de prueba
        all_embeddings = []
        all_labels = []

        best_model.eval()
        with torch.no_grad():
            for batch in data_module.test_dataloader():
                if isinstance(batch, tuple):
                    images, labels = batch
                else:
                    images = batch
                    labels = None

                images = images.to(device)

                # Extraer embeddings
                if hasattr(best_model, 'get_embedding'):
                    embeddings = best_model.get_embedding(images)
                elif hasattr(best_model, 'model') and hasattr(best_model.model, 'get_embedding'):
                    embeddings = best_model.model.get_embedding(images)
                else:
                    if hasattr(best_model, 'model'):
                        logits, embeddings = best_model.model(images)
                    else:
                        logits, embeddings = best_model(images)

                all_embeddings.append(embeddings.cpu().numpy())
                if labels is not None:
                    all_labels.append(labels.cpu().numpy())

        all_embeddings = np.concatenate(all_embeddings, axis=0)
        all_labels = np.concatenate(all_labels, axis=0) if all_labels else None

        print(f"Embeddings extraídos: {all_embeddings.shape}")

        # Aplicar DBSCAN
        dbscan_config = cfg.dbscan if cfg else {
            "eps": 0.5,
            "min_samples": 5,
            "use_pca": True,
            "pca_components": 50,
            "use_tsne": True,
            "tsne_components": 2,
            "tsne_perplexity": 30
        }

        dbscan_results = dbscan_analysis(
            embeddings=all_embeddings,
            eps=dbscan_config.get("eps", 0.5),
            min_samples=dbscan_config.get("min_samples", 5),
            use_pca=dbscan_config.get("use_pca", True),
            pca_components=dbscan_config.get("pca_components", 50),
            use_tsne=dbscan_config.get("use_tsne", True),
            tsne_components=dbscan_config.get("tsne_components", 2),
            tsne_perplexity=dbscan_config.get("tsne_perplexity", 30)
        )

        print(f"\n✓ DBSCAN completado:")
        print(f"  Clusters encontrados: {dbscan_results['n_clusters']}")
        print(f"  Outliers detectados: {dbscan_results['n_noise']}")
        print(f"  Varianza explicada (PCA): {dbscan_results['explained_variance']:.4f}")

        # Visualizar
        save_path = os.path.join(DRIVE_BASE_PATH, 'dbscan_analysis.png')
        visualize_dbscan_results(dbscan_results, labels=all_labels, save_path=save_path)

        # Análisis cuantitativo: Comparar outliers de DBSCAN con ground truth
        if all_labels is not None:
            dbscan_outliers = (dbscan_results['clusters'] == -1).astype(int)
            true_anomalies = all_labels

            # Calcular métricas de clasificación
            dbscan_auc = roc_auc_score(true_anomalies, dbscan_outliers)
            dbscan_ap = average_precision_score(true_anomalies, dbscan_outliers)

            # Calcular métricas adicionales
            from sklearn.metrics import confusion_matrix

            # Matriz de confusión
            cm = confusion_matrix(true_anomalies, dbscan_outliers)
            if cm.size == 4:
                tn, fp, fn, tp = cm.ravel()
            else:
                tn, fp, fn, tp = 0, 0, 0, 0

            # Calcular precisión, recall, F1
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
            accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0.0

            print(f"\n{'='*80}")
            print("ANÁLISIS CUANTITATIVO: DBSCAN vs Ground Truth")
            print(f"{'='*80}")
            print(f"\n📊 Métricas de Clasificación:")
            print(f"  AUC-ROC: {dbscan_auc:.4f}")
            print(f"  Average Precision (AUC-PR): {dbscan_ap:.4f}")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1-Score: {f1_score:.4f}")

            print(f"\n📋 Matriz de Confusión:")
            print(f"                Predicción")
            print(f"              Normal  Anomalía")
            print(f"  Normal      {tn:6d}  {fp:6d}")
            print(f"  Anomalía    {fn:6d}  {tp:6d}")

            print(f"\n📈 Estadísticas de Clusters:")
            print(f"  Total de muestras: {len(all_labels)}")
            print(f"  Muestras normales (ground truth): {np.sum(true_anomalies == 0)}")
            print(f"  Muestras anómalas (ground truth): {np.sum(true_anomalies == 1)}")
            print(f"  Clusters encontrados: {dbscan_results['n_clusters']}")
            print(f"  Outliers detectados por DBSCAN: {dbscan_results['n_noise']}")
            print(f"  Porcentaje de outliers: {dbscan_results['n_noise'] / len(all_labels) * 100:.2f}%")

            # Análisis de distribución de outliers
            normal_outliers = np.sum((dbscan_outliers == 1) & (true_anomalies == 0))
            anomaly_outliers = np.sum((dbscan_outliers == 1) & (true_anomalies == 1))
            normal_in_cluster = np.sum((dbscan_outliers == 0) & (true_anomalies == 0))
            anomaly_in_cluster = np.sum((dbscan_outliers == 0) & (true_anomalies == 1))

            n_normal = np.sum(true_anomalies == 0)
            n_anomaly = np.sum(true_anomalies == 1)

            print(f"\n🔍 Análisis de Distribución:")
            if n_normal > 0:
                print(f"  Normales detectadas como outliers: {normal_outliers} ({normal_outliers/n_normal*100:.2f}% de normales)")
                print(f"  Normales en clusters: {normal_in_cluster} ({normal_in_cluster/n_normal*100:.2f}% de normales)")
            if n_anomaly > 0:
                print(f"  Anomalías detectadas como outliers: {anomaly_outliers} ({anomaly_outliers/n_anomaly*100:.2f}% de anomalías)")
                print(f"  Anomalías en clusters: {anomaly_in_cluster} ({anomaly_in_cluster/n_anomaly*100:.2f}% de anomalías)")

            print(f"\n💡 Interpretación:")
            if dbscan_auc > 0.7:
                print(f"  ✓ DBSCAN muestra buena capacidad para detectar anomalías (AUC-ROC > 0.7)")
            elif dbscan_auc > 0.5:
                print(f"  ⚠️ DBSCAN tiene capacidad moderada para detectar anomalías (0.5 < AUC-ROC < 0.7)")
            else:
                print(f"  ❌ DBSCAN tiene capacidad limitada para detectar anomalías (AUC-ROC < 0.5)")

            if n_anomaly > 0 and anomaly_outliers / n_anomaly > 0.5:
                print(f"  ✓ Más del 50% de las anomalías fueron detectadas como outliers")
            elif n_anomaly > 0:
                print(f"  ⚠️ Menos del 50% de las anomalías fueron detectadas como outliers")

            print(f"{'='*80}\n")
    else:
        print("⚠️ No se encontró el mejor modelo")
else:
    print("⚠️ No hay modelos entrenados para analizar")


NameError: name 'best_3_models' is not defined

## 9. Resumen y Conclusiones

Resumen de resultados y comparación final de todos los modelos.


In [ ]:
# Crear resumen final
print("="*80)
print("RESUMEN FINAL DEL PROYECTO II")
print("="*80)

print(f"\n📊 Modelos entrenados:")
if 'model_a_results' in globals():
    print(f"  - Modelo A (CNN desde cero): {len(model_a_results)} configuraciones")
if 'model_b_results' in globals():
    print(f"  - Modelo B (CNN con destilación): {len(model_b_results)} configuraciones")
if 'model_c_results' in globals():
    print(f"  - Modelo C (Autoencoder U-Net): {len(model_c_results)} configuraciones")

if 'best_3_models' in globals() and best_3_models:
    print(f"\n🏆 Top 3 modelos (por AUC-ROC):")
    for i, result in enumerate(best_3_models, 1):
        best_auc = max(result.get("auc_roc", 0), result.get("auc_roc_mah", 0), result.get("auc_roc_recon", 0))
        print(f"  {i}. {result['model_type']} - {result['config']}: AUC-ROC={best_auc:.4f}")

if 'quantization_results' in globals() and quantization_results:
    print(f"\n⚡ Cuantización (3 mejores modelos):")
    for result in quantization_results:
        print(f"  - {result['model_type']} - {result['config']}:")
        print(f"    Compresión: {result['compression_ratio']:.2f}x")
        print(f"    Speedup: {result['speedup']:.2f}x")
        print(f"    Retención AUC-ROC: {result.get('performance_retention_auc_roc', 0):.2f}%")
        print(f"    Retención AUC-PR: {result.get('performance_retention_auc_pr', 0):.2f}%")

print(f"\n✓ Proyecto completado exitosamente")
print(f"  Revisa los resultados en WandB: https://wandb.ai")
print(f"  Proyecto: proyecto-ii-anomaly-detection")
print("="*80)


## 10. Notas Finales

### Instrucciones de Uso

1. **Ejecutar en orden**: Ejecuta las celdas en orden secuencial desde el inicio
2. **Google Drive**: Asegúrate de tener montado Google Drive y que contenga:
   - El dataset MVTec AD en la ruta especificada
   - Los archivos de configuración en `conf/` (opcional, se crean automáticamente si no existen)
3. **WandB**: Necesitas autenticarte con WandB cuando se ejecute `wandb.login()`
4. **Tiempo de ejecución**: El entrenamiento completo puede tardar varias horas dependiendo del hardware

### Estructura del Código

Todo el código está incluido en este notebook:
- ✅ **Sección 1**: Definición de modelos (BasicBlock, CNNClassifier, UNetAutoencoder)
- ✅ **Sección 2**: Módulos Lightning (CNNClassifierLightning, AutoencoderLightning, LossFunctions)
- ✅ **Sección 3**: Configuración Hydra
- ✅ **Sección 4**: DataModule (MVTecDataModule, AnomalyDataset)
- ✅ **Sección 5**: Entrenamiento de modelos
- ✅ **Sección 6**: Evaluación de anomalías
- ✅ **Sección 7**: Cuantización
- ✅ **Sección 8**: Análisis DBSCAN
- ✅ **Sección 9**: Resumen final

### Resultados

Todos los resultados se guardan en:
- **WandB**: Métricas, gráficas y visualizaciones
- **Google Drive**: Checkpoints de modelos, imágenes y análisis
- **Consola**: Resúmenes y métricas principales
